In [1]:
# 在类的构造方法中定义属性时，默认会把属性名和值当做键值对放在一个字典中，可以通过该对象的
# __dict__查看，当我们相对属性赋值操作进行拦截时，可以自定义__setattr__方法实现

In [3]:
class Student(object):
    def __init__(self, name, age):
        self.name = name
        self.age = age

s = Student("zhangsan", 18)
print(s.__dict__)


{'name': 'zhangsan', 'age': 18}


In [4]:
# 要注意__setattr__内不要有赋值操作，否则就会出现无线递归调用__setattr__的情况，
# 应该使用使用self.__dict__['name'] = value.
class Student(object):
    def __init__(self, name, age):
        self.name = name
        self.age = age

    def __setattr__(self, key, value):
        print("preprocess")
        print("you define an attribute, its name is %s and value is %s" % (key, value))
        self.__dict__[key] = value
        print("postprocess")

s = Student("zhangsan", 18)
print(s.__dict__)

# 像这种拦截操作对数据进行preprocess和postprocess的例子有很多，中间件就是这样一种思想
# 同样在pytorch中，对tensor进行forward计算前后，会先调用hook函数对tensor进行操作

you define an attribute, its name is name and value is zhangsan
postprocess
you define an attribute, its name is age and value is 18
postprocess
{'name': 'zhangsan', 'age': 18}


In [16]:
# 我们利用__setattr__方法可以构建一类对象的数据结构

class Ancestor(object):
    def __init__(self):
        self.container = {}

    def __setattr__(self, key, value):
        self.__dict__[key] = value
        self.__dict__['container'][key] = value 

class Son(Ancestor):
    def __init__(self, name):
        super(Son,self).__init__()
        self.name = name

class Father(Ancestor):
    def __init__(self):
        super(Father, self).__init__()
        self.child = Son("xiaoming")


class GrandFather(Ancestor):
    def __init__(self):
        super(GrandFather, self).__init__()
        self.child = Father()

grand_father = GrandFather()
print(grand_father.__dict__)
# print("23")
# grand_father = GrandFather()
# print("test")
# print(grand_father.container)
# print("over")

{'container': {'container': {...}, 'child': <__main__.Father object at 0x0000026C22EEC0D0>}, 'child': <__main__.Father object at 0x0000026C22EEC0D0>}
